In [96]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import pandas as pd
import os
from scipy.stats import norm

In [17]:
ruta_entrada= os.path.join( '.','data')
ruta_salida = os.path.join( '.','output')
ciudad = 'Valledupar'
fuente = 'IDEAM'
ruta_serie= os.path.join(ruta_entrada,ciudad,fuente+'.csv')
serie= pd.read_csv(ruta_serie, sep = ';',
                encoding='latin-1')

## Punto 1

In [3]:
def ENFICC(serie_horaria):  #Cálculo ENFICC 99%
    agnos=sorted(serie_horaria['AÑO'].unique())
    ghi_df=pd.DataFrame(columns=["AÑO","MES","DIAS","GHImes"])
    energia = list(serie_horaria)[-1]
    for agno in agnos:
        for mes in range(1,13):
            dias = sorted(serie_horaria[(serie_horaria['AÑO'] == agno) &
                        (serie_horaria['MES'] == mes)]["DIA"].unique())
            ghi_mes = serie_horaria.loc[((serie_horaria["AÑO"] == agno)
                            & (serie_horaria["MES"] == mes)),
                                        energia].sum()/1000
            ghi_dict = {"AÑO":agno, "MES":mes, "DIAS":dias[-1],
                        "GHImes":ghi_mes}
            ghi_df = ghi_df.append(ghi_dict, ignore_index=True)
        
        
    agnos2=sorted(ghi_df['AÑO'].unique())
    df_dias=pd.DataFrame(columns=["AÑO","MES","GHIdia"])
    for agno in agnos2:
        for mes in range(1,13):
            dia=ghi_df.loc[((ghi_df['AÑO']==agno) &
                                   (ghi_df['MES']==mes)),'DIAS'].item()
            ghi_diario=ghi_df.loc[((ghi_df['AÑO']==agno) & 
                                          (ghi_df['MES']==mes)),'GHImes'].item()/dia
            ghi_dict = {'AÑO':agno,'MES':mes,'GHIdia':ghi_diario}
            df_dias=df_dias.append(ghi_dict,ignore_index=True)
        
    ENFICC=df_dias.quantile(0.01,numeric_only=True)['GHIdia']
        
    return print('La ENFICC 99% es', ENFICC)

## Punto 3

In [4]:
corta=serie[serie['AÑO'] == 2010] #Df con todo el año 2010
seriecorta2=serie[(serie['AÑO'] >= 2007) & (serie['AÑO'] <=2012 )] #DF con año 2007 al 2011

In [5]:
ciudad = 'Valledupar'
fuente2 = 'NREL'
ruta_serie= os.path.join(ruta_entrada,ciudad,fuente2+'.csv')
serienrel= pd.read_csv(ruta_serie, sep = ';',
                encoding='latin-1')

In [131]:
def MCP(serie_corta,serie_larga):
    serie_ajustada=pd.DataFrame(columns=['AÑO','MES','DIA','HORA','GHI'])
    ghi_obj=serie_corta['IDEAM'].sum()/len(serie_corta)
    sigma_obj=serie_corta['IDEAM'].std()
    serie_larga_comun=serie_larga[(serie_larga['AÑO']>=(serie_corta.iloc[0,0])) & (serie_larga['AÑO']<=(serie_corta.iloc[-1,0]))]
    ghi_ref=serie_larga_comun['NREL'].sum()/len(serie_larga_comun)
    sigma_ref=serie_larga_comun['NREL'].std()
    for i in range(0,len(serie_larga)):
        ghi_i=((ghi_obj-(sigma_obj/sigma_ref)*ghi_ref)+(sigma_obj/sigma_ref)*serie_larga.iloc[i,4])
        if serie_larga.iloc[i,4]==0:
            ghi_i=0
            
        ano=serie_larga.iloc[i,0]
        mes=serie_larga.iloc[i,1]
        dia=serie_larga.iloc[i,2]
        hora=serie_larga.iloc[i,3]
        ghi_dict = {"AÑO":ano, "MES":mes, "DIA":dia,'HORA':hora,
                        "GHI":ghi_i}
        serie_ajustada = serie_ajustada.append(ghi_dict, ignore_index=True)
    return serie_ajustada

In [132]:
MCP(corta,serienrel)

,AÑO,MES,DIA,HORA,GHI
0,2005,1,1,0,0
1,2005,1,1,1,0
2,2005,1,1,2,0
3,2005,1,1,3,0
4,2005,1,1,4,0
5,2005,1,1,5,0
6,2005,1,1,6,4.76198
7,2005,1,1,7,153.921
8,2005,1,1,8,358.959
9,2005,1,1,9,544.17


## Punto 4

#### MBE

In [64]:
def MBE(serie_mod,serie_obs):
    serie_mod2=serie_mod[(serie_mod['AÑO'] >=serie_obs.iloc[0,0])]
    error=0
    sumatoria=0
    for i in range(0,len(serie_obs)):
        resta=serie_obs.iloc[i,4]-serie_mod2.iloc[i,4]
        error=error+resta
        sumatoria=sumatoria+serie_obs.iloc[i,4]
    resultado=error/sumatoria
    return print('El MBE es:',"{:.2%}".format(resultado) )

In [106]:
def RMSE(serie_mod,serie_obs):
    serie_mod2=serie_mod[(serie_mod['AÑO'] >=serie_obs.iloc[0,0])]
    error=0
    for i in range(0,len(serie_obs)):
        resta=(serie_obs.iloc[i,4]-serie_mod2.iloc[i,4])**2
        error=error+resta
    error=sqrt(error/len(serie_obs))
    resultado=error/(serie_obs['IDEAM'][serie_obs['IDEAM'].idxmax()]-serie_obs['IDEAM'][serie_obs['IDEAM'].idxmin()])
    return print('El RMSE es:',"{:.2%}".format(resultado) )

In [108]:
RMSE(serienrel,serie)

El RMSE es: 9.07%


In [112]:
norm.cdf(6)

0.9999999990134123